In [1]:
#hide
#skip
! [ -e /content ] && pip install -Uqq fastai # upgrade fastai on colab

In [ ]:
# import torch; torch.cuda.device_count()
# import torch; torch.cuda.get_device_properties(0)

In [1]:
%load_ext autoreload
%autoreload 1

# MIMIC III (sample) Code Prediction

In [2]:
import fastbook
fastbook.setup_book()

from fastbook import *

Application specific imports from fastai:

In [3]:
from fastai.text.all import *
from fastai.distributed import *

xcube specific imports:

In [4]:
from xcube.text.learner import text_classifier_learner
from xcube.metrics import PrecisionK

#### Setting the base path of the repository

In [5]:
path = Path.cwd()
path_data = path/'data'
path_model = path/'models'

path, path_data, path_model

(Path('/home/deb/xcube/nbs/examples/mimic'),
 Path('/home/deb/xcube/nbs/examples/mimic/data'),
 Path('/home/deb/xcube/nbs/examples/mimic/models'))

## `DataLoaders` for the Multi-Label Classifier (frozen)

In [7]:
!ls -lh {path_data/"*caml*"}

-rw-r--r-- 1 deb deb 472M Feb  3 19:08 /home/deb/xcube/nbs/examples/mimic/data/caml_notes_labelled.csv
-rw-r--r-- 1 deb deb  48M Feb  4 16:28 /home/deb/xcube/nbs/examples/mimic/data/caml_notes_labelled_sample_10percent.csv


In [8]:
!ls -lh {path_model/"caml*recent*"}

-rw-r--r-- 1 deb deb 1.2G Feb  5 20:41 /home/deb/xcube/nbs/examples/mimic/models/caml_dls_lm_recent.pkl
-rw-r--r-- 1 deb deb 909K Feb  5 20:41 /home/deb/xcube/nbs/examples/mimic/models/caml_dls_lm_vocab_recent.pkl
-rw-r--r-- 1 deb deb 157M Feb  6 02:29 /home/deb/xcube/nbs/examples/mimic/models/caml_lm_finetuned_recent.pth
-rw-r--r-- 1 deb deb 157M Feb  6 00:43 /home/deb/xcube/nbs/examples/mimic/models/caml_lm_recent.pth


In [9]:
data_file = path_data/'caml_notes_labelled_sample_10percent.csv'
lm_vocab_file = path_model/'caml_dls_lm_vocab.pkl'

In [10]:
df = pd.read_csv(data_file, dtype={'text': str, 'labels': str})
df[['text', 'labels']] = df[['text', 'labels']].astype('str')

In [11]:
len(df)

5273

In [16]:
# df.head(3)

---

In [18]:
df['length'] = df.apply(lambda row: len(row.text), axis=1)

In [ ]:
df

In [ ]:
np.percentile(df.length, 50)

In [ ]:
np.mean(df.length)

In [21]:
df['text'] = df.apply(lambda row: row.text[10000:], axis=1)

In [ ]:
df

In [ ]:
print(df.text[678])

---

In [12]:
lm_vocab = torch.load(lm_vocab_file)
L(lm_vocab)

(#57376) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the'...]

In [13]:
label_freq = Counter()
for labels in df.labels: label_freq.update(labels.split(';'))
lbls = list(label_freq.keys())

In [14]:
L(lbls)

(#4716) ['518.81','348.4','348.82','801.25','427.89','E882','V49.86','305.00','96.71','38.93'...]

In [15]:
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].tolist()
    return train, valid

In [16]:
train, valid = splitter(df)

In [17]:
L(train), L(valid)

((#4940) [0,1,2,3,4,5,6,7,8,9...],
 (#333) [4940,4941,4942,4943,4944,4945,4946,4947,4948,4949...])

In [18]:
dblock = DataBlock(
        blocks = (TextBlock.from_df('text', seq_len=72, vocab=lm_vocab), MultiCategoryBlock(vocab=lbls)),
        get_x = ColReader('text'),
        get_y = ColReader('labels', label_delim=';'),
        splitter = splitter,
)

In [19]:
dls = dblock.dataloaders(df, bs=32)

In [20]:
torch.save(dls, path_model/'caml_dls_clas_sample_10percent_32.pkl')

---

In [24]:
dls_clas = torch.load(path_model/'caml_dls_clas_sample_10percent_32.pkl')

In [25]:
dls_clas.show_batch(max_n=1)

/home/deb/miniconda3/lib/python3.9/site-packages/fastai/torch_core.py:475: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/deb/miniconda3/lib/python3.9/site-packages/fastai/torch_core.py:475: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))


,text,None
0,xxbos admission date discharge date date of birth sex m service medicine allergies penicillins sulfa sulfonamides attending first name3 lf chief complaint lethargy persistent sinus tachycardia major surgical or invasive procedure none history of present illness mr known lastname is a year old male with hiv cd4 one month ago parkinson s disease htn chronic lower extremity edema and recurrent cellulitis who was brought to ed from hospital1 with lethargy and leukocytosis and then admitted to micu with colitis and persistent sinus tachycardia patient is somewhat unreliable historian but reports weeks of diarrhea and abdominal pain that have been worsening he reports coming to the hospital because he continued to ask and express concern about his diarrhea per hospital1 notes has been having loose bowel movements x6 with wbcs of 24k t refusing meals and lethargic of note he was treated with course of ciprofloxacin for uti in in the,427.89;584.9;995.92;785.52;599.0;305.1;V12.04;008.45;296.80;348.39;041.6;042;038.8;332.0;304.01;304.21


In [26]:
coll_repr(dls_clas.vocab[0], 30)

"(#51928) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','and','to','of','was','with','a','on','in','for','mg','no','tablet','patient','is','he','at','blood','name','po','she'...]"

In [27]:
coll_repr(dls_clas.vocab[1], 30)

"(#4716) ['518.81','348.4','348.82','801.25','427.89','E882','V49.86','305.00','96.71','38.93','861.32','958.4','868.14','881.20','289.82','862.1','902.53','860.5','991.6','E922.9','V66.7','32.39','34.02','38.84','54.11','34.82','39.32','34.09','86.28','99.04'...]"

In [28]:
# x, y = dls_clas.one_batch()
# x.shape, y.shape

In [29]:
# x, y = first(dls_clas.train)
dls_iter = iter(dls_clas.train)

In [32]:
x,y = next(dls_iter)

In [33]:
x.shape, y.shape

(torch.Size([32, 3560]), torch.Size([32, 4716]))

In [34]:
coll_repr(lm_vocab, 30)

"(#51928) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','the','and','to','of','was','with','a','on','in','for','mg','no','tablet','patient','is','he','at','blood','name','po','she'...]"

In [35]:
' '.join(lm_vocab[o] for o in x[0][:1000])

'xxbos admission date discharge date date of birth sex m service medicine allergies sevoflurane location un juice reglan bactrim attending first name3 lf chief complaint fevers chills nausea vomiting inability to take po major surgical or invasive procedure graft excision tunnelled hd catheter placement history of present illness this is a yo m with dm1 c b nephropathy esrd on hd through av graft tts and esophagitis who presents with three days of fevers chills nausea vomiting and inability to tolerate po the patient was in his usoh until saturday when he developed intractable nausea after hd the patient notes a normal hd session only complicated by a small amount of bleeding at the graft site after hd the patient had nausea and vomited some food bilious material and what he says is coffee ground color material the patient did not have any hematemesis he vomited times and was unable to eat or drink the patient denied any other gi symptoms including diarrhea melena or hematochezia he en

## `DataLoaders` for the Multi-Label Classifier (using fastai's Mid-Level Data API):

### Loading Raw Data

In [7]:
data_file = path_data/'caml_notes_labelled_sample_10percent.csv'
lm_vocab_file = path_model/'caml_dls_lm_vocab.pkl'

In [16]:
df = pd.read_csv(data_file, dtype={'text': str, 'labels': str})
df[['text', 'labels']] = df[['text', 'labels']].astype('str')

In [17]:
df.head(3)

,subject_id,hadm_id,text,labels,is_valid
0,30088,172719,"Admission Date: [**2179-3-7**] Discharge Date: [**2179-3-18**]\n\nDate of Birth: [**2121-12-23**] Sex: M\n\nService: MEDICINE\n\nAllergies:\nIbuprofen\n\nAttending:[**First Name3 (LF) 613**]\nChief Complaint:\ndyspnea, hyperkalemia\n\nMajor Surgical or Invasive Procedure:\nHemodialysis\nright Femoral central venous line placement and removal\n\n\nHistory of Present Illness:\n57 y/o M with hx of ESRD presents after intentionally missing\ndialysis for last 9 days. Came to the ED due to the coaxing of\nhis wife. Complains of SOB and DOE lasting about the last ...",276.7;585.6;348.39;790.7;427.32;507.0;403.10;825.0;E888.9;300.9;301.9;250.62;357.2;250.42;285.21;278.01;427.31;250.82;V15.81;276.52;707.07;707.22;041.19;39.95;38.93,False
1,22624,138579,"Admission Date: [**2117-1-21**] Discharge Date: [**2117-1-28**]\n\nDate of Birth: [**2070-8-1**] Sex: M\n\nService: MEDICINE\n\nAllergies:\nPatient recorded as having No Known Allergies to Drugs\n\nAttending:[**First Name3 (LF) 943**]\nChief Complaint:\nBright red blood in stools\n\nMajor Surgical or Invasive Procedure:\nUpper endoscopy\nColonoscopy\nTransfusion of platelets and fresh frozen plasma\n\n\nHistory of Present Illness:\nIn brief, a 46-year-old man with etoh cirrhosis, varices s/p\nbanding in [**12-9**], HCV not on treatment, who initially presente...",455.5;303.00;571.2;291.81;070.70;284.8;286.7;562.10;571.1;211.2;456.21;305.1;276.8;45.13;45.23;99.05;99.07;96.34,False
2,93593,164601,Admission Date: [**2161-5-8**] Discharge Date: [**2161-5-12**]\n\n\nService: MEDICINE\n\nAllergies:\nPercocet\n\nAttending:[**First Name3 (LF) 1115**]\nChief Complaint:\nGI bleed / coffee-ground emesis.\n\nMajor Surgical or Invasive Procedure:\nEGD\nBlood transfusion\n\n\nHistory of Present Illness:\nPatient is an 87-y/o woman with a history of colon cancer s/p\nright colectomy who presented today to the emergency room after\n2 episodes of coffee-ground emesis at her rehab facility.\nHistory was taken from family members as patient was in\nsignificant discomfort with the NG...,V10.05;V45.72;V15.82;788.30;531.40;998.59;682.6;438.89;728.87;458.29;564.09;401.9;285.9;294.8;311;272.0;E878.2;45.13,True


let's now gather the labels from the 'labels' column of the df:

In [18]:
label_freq = Counter()
for labels in df.labels: label_freq.update(labels.split(';'))

The total number of labels are:

In [19]:
len(label_freq)

4716

Let's make a list of all labels (We will use it later while creating the `DataLoader`)

In [20]:
lbls = list(label_freq.keys()) 

### Steps for creating the classifier `DataLoaders` using fastai `Transforms`:

#### 1. train/valid `splitter`:

Okay, based on the `is_valid` column of our Dataframe, let's create a splitter:

In [21]:
def splitter(df):
    train = df.index[~df['is_valid']].tolist()
    valid = df.index[df['is_valid']].to_list()
    return train, valid

Let's check the train/valid split

In [22]:
splits = [train, valid] = splitter(df)
L(splits[0]), L(splits[1])

((#4940) [0,1,3,4,5,6,8,9,10,11...], (#333) [2,7,13,16,21,26,38,43,61,63...])

#### 2. Making the `Datasets` object:

**Crucial:** We need the vocab of the language model so that we can make sure we use the same correspondence of token to index. Otherwise, the embeddings we learned in our fine-tuned language model won't make any sense to our classifier model, and the fine-tuning won't be of any use. So we need to pass the `lm_vocab` to the `Numericalize` transform:

So let's load the vocab of the language model:

In [23]:
lm_vocab = torch.load(lm_vocab_file)

In [24]:
L(lm_vocab)

(#60008) ['xxunk','xxpad','xxbos','xxeos','xxfld','xxrep','xxwrep','xxup','xxmaj','*'...]

In [16]:
@Transform
def Shrink(num:TensorText): return num[:2500] if len(num)>2500 else num

In [25]:
x_tfms = [Tokenizer.from_df('text'), attrgetter("text"), Numericalize(vocab=lm_vocab)] 
y_tfms = [ColReader('labels', label_delim=';'), MultiCategorize(vocab=lbls), OneHotEncode()]
tfms = [x_tfms, y_tfms]

---

In [17]:
# class Shrink(Transform):
#     def encodes(self, num):
#         return num[:3500] if len(num)>3500 else num

In [18]:
# tls_x = TfmdLists(df, x_tfms, splits=splits)

---

In [26]:
dsets = Datasets(df, tfms, splits=splits)

Let's now check if our `Datasets` got created alright:

In [27]:
len(dsets.train), len(dsets.valid)

(4940, 333)

In [20]:
dsets.train[0]

(TensorText([    2,    50,    51,    30,    51,    51,    12,   225,   248,   321,   106,   174,   157,    20,    95,   157,  1920,   549,  1934,   301,    60,   271,   227,   324,   309,    32,    62,
           781,   311,   175,    31,   313,   173,   351,    41,    12,   137,   234, 20920,   486,    32,    62,   781,    61,  4261,    25,  2066,   228,  1290,    10,  6786,   896,    16,  4461,
           350,  3732,  4343,    18,  6340,   334,    80,    17,     9,  1361,     9,    22,    13,   228,    11,    48,  3281,    11,     9,    32,    14,   275,    12,   218,    29,    13,    86,
          3262,  1247,    10,    13,    98,    16,  1462,   129,   123,    41,   882,   238,    41,   882,   147,    41,   882,   145,   138,   934,   335,   223,    66,   190,   293,  3688,    12,
             9,    39, 10039,  5654,  5811,   604,    18,  1017,  5205,  2213,   226,  1912,    12,     9,    39,  1160,   579,   160,  6685, 10040,  5796,   689,   759,   358,   319,    47,    48,
      

---

In [21]:
type(dsets.train[0][0])

fastai.text.data.TensorText

In [22]:
len(dsets.train[0][0])

361

In [33]:
torch.arange(10)

tensor([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [36]:
dsets.train[0][0][:] = TensorText(torch.arange(361))

In [37]:
dsets.train[0][0]

TensorText([    2,    50,    51,    30,    51,    51,    12,   225,   248,   321,   106,   174,   157,    20,    95,   157,  1920,   549,  1934,   301,    60,   271,   227,   324,   309,    32,    62,
          781,   311,   175,    31,   313,   173,   351,    41,    12,   137,   234, 20920,   486,    32,    62,   781,    61,  4261,    25,  2066,   228,  1290,    10,  6786,   896,    16,  4461,
          350,  3732,  4343,    18,  6340,   334,    80,    17,     9,  1361,     9,    22,    13,   228,    11,    48,  3281,    11,     9,    32,    14,   275,    12,   218,    29,    13,    86,
         3262,  1247,    10,    13,    98,    16,  1462,   129,   123,    41,   882,   238,    41,   882,   147,    41,   882,   145,   138,   934,   335,   223,    66,   190,   293,  3688,    12,
            9,    39, 10039,  5654,  5811,   604,    18,  1017,  5205,  2213,   226,  1912,    12,     9,    39,  1160,   579,   160,  6685, 10040,  5796,   689,   759,   358,   319,    47,    48,
         12

---

In [28]:
dsets.decode(dsets.train[0])

("xxbos xxmaj admission xxmaj date : [ * * 2179 - 3 - 7 * * ] xxmaj discharge xxmaj date : [ * * 2179 - 3 - 18 * * ] \n\n xxmaj date of xxmaj birth : [ * * 2121 - 12 - 23 * * ] xxmaj sex : m \n\n xxmaj service : xxup medicine \n\n xxmaj allergies : \n xxmaj ibuprofen \n\n attending:[**first xxmaj name3 ( lf ) 613 * * ] \n xxmaj chief xxmaj complaint : \n dyspnea , hyperkalemia \n\n xxmaj major xxmaj surgical or xxmaj invasive xxmaj procedure : \n xxmaj hemodialysis \n right xxmaj femoral central venous line placement and removal \n\n\n xxmaj history of xxmaj present xxmaj illness : \n 57 y / o m with hx of xxup esrd presents after intentionally missing \n dialysis for last 9 days . xxmaj came to the xxup ed due to the coaxing of \n his wife . xxmaj complains of xxup sob and xxup doe lasting about the last five \n days . xxmaj also has cough with clear sputum production . xxmaj also \n complains of bilateral pedal edema , xxup r > l , starting around the \n time xxup sob started . xxmaj

Looks fine!

#### 3. Making the `DataLoaders` object:

We need to pick the sequence length and the batch size:

In [29]:
bs, sl = 16, 72

We will use the `dl_type` argument of the `DataLoaders`. The purpose is to tell `DataLoaders` to use `SortedDL` class of the `DataLoader`, and not the usual one. `SortedDL` constructs batches by putting samples of roughly the same lengths into batches.

In [30]:
dl_type = partial(SortedDL, shuffle=True)

**Crucial:** 
- We will use **`pad_input_chunk`** because our encoder `AWD_LSTM` will be wrapped inside `SentenceEncoder`. 
- A `SenetenceEncoder` expects that all the documents are padded, 
- with most of the padding at the beginning of the document, with each sequence beginning at a round multiple of bptt
- and the rest of the padding at the end.

In [31]:
dls_clas = dsets.dataloaders(bs=bs, seq_len=sl, 
                        dl_type=dl_type,
                       before_batch=pad_input_chunk)

Creating the `DataLoaders` object takes considerable amount of time, so let's save it:

In [23]:
torch.save(dls_clas, path_model/'caml_dls_clas_sample_10percent_16.pkl')

Let's take a look at the data:

In [32]:
dls_clas.show_batch(max_n=3)

/home/deb/miniconda3/lib/python3.9/site-packages/fastai/torch_core.py:475: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/deb/miniconda3/lib/python3.9/site-packages/fastai/torch_core.py:475: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/deb/miniconda3/lib/python3.9/site-packages/fastai/torch_core.py:475: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/home/deb/miniconda3/lib/python3.9/site-packages/fastai/torch_core.py:475: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  ax = ax.append(pd.Series({label: o}))
/hom

,text,None
0,"xxbos xxmaj admission xxmaj date : [ * * 2127 - 5 - 12 * * ] xxmaj discharge xxmaj date : [ * * 2127 - 5 - 21 * * ] \n\n xxmaj date of xxmaj birth : [ * * 2074 - 1 - 7 * * ] xxmaj sex : m \n\n xxmaj service : xxup medicine \n\n xxmaj allergies : \n xxmaj penicillins / xxmaj sulfa ( sulfonamides ) \n\n attending:[**first xxmaj name3 ( lf ) 4028 * * ] \n xxmaj chief xxmaj complaint : \n xxmaj lethargy , persistent sinus tachycardia . \n\n xxmaj major xxmaj surgical or xxmaj invasive xxmaj procedure : \n xxmaj none . \n\n xxmaj history of xxmaj present xxmaj illness : \n xxmaj mr . [ * * known lastname xxunk * * ] is a 53 year old male with xxup hiv ( cd4 352 one month",348.39;305.1;427.89;584.9;599.0;008.45;995.92;041.6;296.80;785.52;V12.04;332.0;042;304.01;038.8;304.21
1,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,427.31;39.95;38.93;45.13;414.01;96.72;518.81;428.0;244.9;99.04;535.50;584.9;250.00;88.72;38.95;038.0;008.45;999.9;31.1;96.6;412;V45.82;43.11;493.90;995.92;255.41;276.0;14.0;428.32;287.4;593.9;482.1;790.5;078.5;459.0
2,xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad xxpad,348.39;427.32;427.31;276.8;V10.05;V15.82;401.9;272.0;414.01;300.00;599.0;96.6;238.71;V45.82;345.90;V58.61;438.11;041.04;432.1;13.1;518.4


In [24]:
dls_clas = torch.load(path_model/'caml_dls_clas_sample_10percent_32.pkl')

---

In [41]:
dls_iter = iter(dls_clas.train)

In [67]:
x, y = next(dls_iter)

In [68]:
x[23].shape

torch.Size([1106])

---

## `Learner` for Multi-Label Classifier Fine-Tuning

In [6]:
dls_clas = torch.load(path_model/'caml_dls_clas_sample_10percent_16.pkl')

In [7]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.1, max_len=72*40, metrics=PrecisionK).to_fp16()

In [8]:
learn = learn.load_encoder(path_model/'caml_lm_finetuned')

In [9]:
decoder = (learn.model[1].__class__)
decoder??

Init signature: decoder(dims, ps, bptt, y_range=None)
Docstring:      Same as `nn.Module`, but no need for subclasses to call `super().__init__`
Source:        
class LabelAttentionClassifier(Module):
    def __init__(self, dims, ps, bptt, y_range=None):
        self.lbs = dims[-1]
        self.fts = dims[0]//3
        # self.layers = LinBnDrop(self.lbs, self.fts, p=ps, act=None) # wrong_deb

        # ps = 0.1 # deb
        self.layers = LinBnDrop(self.lbs, ln=False, p=ps, act=None) # deb
        self.bptt = bptt
        self.emb_label = Embedding(self.lbs, self.fts) # deb
        self.final_lin = nn.Linear(self.fts, self.lbs)

    def forward(self, input):
        out, _ = input
        # x = masked_concat_pool(out, mask, self.bptt)

        bs = out.shape[0]
        # ctx = out.new_zeros((bs, self.lbs, self.fts))
        # for out_split in torch.split(out, 1, dim=1):
        attn_wgts = out @ self.emb_label.weight.transpose(0, 1) # deb
        attn_wgts = F.softmax(attn_wgts, 1) # d

## Fine-Tuning the Classifier 

In [27]:
# lr_min, lr_steep, lr_valley, lr_slide = learn.lr_find(suggest_funcs=(minimum, steep, valley, slide))
# lr_min, lr_steep, lr_valley, lr_slide

In [10]:
learn.fit_one_cycle(1, lr_max=1e-1, moms=(0.8,0.7,0.8), wd=0.1)

epoch,train_loss,valid_loss,PrecisionK,time
0,0.015008,0.015344,0.400400,01:49


In [11]:
learn.freeze_to(-2)

In [30]:
learn.fit_one_cycle(8, lr_max=1e-2, moms=(0.8,0.7,0.8), wd=0.1, cbs=SaveModelCallback(fname=path_model/'caml_clas_sample_10percent'))

epoch,train_loss,valid_loss,PrecisionK,time
0,0.012916,0.015606,0.372973,02:50
1,0.012462,0.014843,0.401001,02:50
2,0.011493,0.014297,0.422823,02:50
3,0.010979,0.014140,0.443644,02:50
4,0.010012,0.013461,0.463463,02:49
5,0.008947,0.013103,0.480681,02:49
6,0.008049,0.012796,0.487087,02:45
7,0.007513,0.012722,0.490891,02:49


Better model found at epoch 0 with valid_loss value: 0.015605793334543705.
Better model found at epoch 1 with valid_loss value: 0.014843019656836987.
Better model found at epoch 2 with valid_loss value: 0.014297299087047577.
Better model found at epoch 3 with valid_loss value: 0.014140406623482704.
Better model found at epoch 4 with valid_loss value: 0.01346144825220108.
Better model found at epoch 5 with valid_loss value: 0.01310289092361927.
Better model found at epoch 6 with valid_loss value: 0.012796320021152496.
Better model found at epoch 7 with valid_loss value: 0.012722250074148178.


In [13]:
learn.freeze_to(-3)

In [14]:
learn.fit_one_cycle(10, lr_max=1e-3, moms=(0.8,0.7,0.8), wd=0.4, cbs=SaveModelCallback(fname=path_model/'caml_clas_sample_10percent'))

epoch,train_loss,valid_loss,PrecisionK,time
0,0.007250,0.012783,0.486486,03:46
1,0.007518,0.012881,0.486887,03:46
2,0.007975,0.012882,0.481281,03:47
3,0.008674,0.012922,0.481682,03:46
4,0.008664,0.012887,0.484284,03:48
5,0.008329,0.013090,0.487087,03:46
6,0.008283,0.012964,0.487688,03:45
7,0.008249,0.013158,0.491491,03:45
8,0.008288,0.012827,0.492693,03:45
9,0.007783,0.012901,0.492693,03:44


Better model found at epoch 0 with valid_loss value: 0.012782841920852661.


In [15]:
learn.unfreeze()

In [16]:
learn.fit_one_cycle(5, lr_max=1e-4, moms=(0.8,0.7,0.8), wd=0.7)

epoch,train_loss,valid_loss,PrecisionK,time
0,0.007327,0.012788,0.488088,04:54
1,0.007428,0.012811,0.490891,04:57
2,0.007452,0.012863,0.489289,05:00
3,0.007476,0.012764,0.489690,05:01
4,0.007513,0.012789,0.489489,05:00


In [ ]:
learn.recorder.plot_loss()

In [ ]:
plt.plot(L(learn.recorder.values).itemgot(2));